In [1]:
# # This cell runs bowtie locally, uncomment last line to run the command and test your bowtie
# 
# import os
# # wget index and SRR043378 directly into project folder
# threads = 4  # Can be changed
# # folder name then file name
# bowtieIndex = "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index/" \
#               "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index"
# # fastq read file
# reads = "SRR043378_1.fastq/data"
# cmd = "bowtie2 -p "+str(threads)+" -x " + bowtieIndex + " -U "+reads+" -S out.sam"
# 
# 
# os.system(cmd)

In [2]:
# import pyspark
# import os
# 
# # new sparkContext
# sc = pyspark.SparkContext(appName="DNA")

In [3]:
# Basis for map function

# threads = 1  # Can be changed
# # folder name then file name
# bowtieIndex = "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index/" \
#               "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bowtie_index"
# # fastq read file
# reads = "SRR043378_1.fastq/data"
# cmd = "bowtie2 -p " + str(threads) + " -x " + bowtieIndex + " -U " + reads + " -S out.sam"
# os.system(cmd)

In [4]:
#import findspark
#findspark.init('/home/taylor/spark')

from pyspark import SparkConf, SparkContext
import math
from pyspark.sql import SQLContext


conf = SparkConf().setMaster("local").setAppName("DNA")

sc = SparkContext(conf=conf)

In [9]:
# creates tuples with format (line number, string)
raw_input = (sc.textFile("SRR043378_1.fastq/data")).zipWithIndex()

In [12]:
# create tuple with format (read number, string)
indexed_raw_input = raw_input.map(lambda x: (math.floor(x[1]/4), x[0]))

In [13]:
# Function to pass to map function that concatenates the 4 strings
def make_reads(iterable_read):
    x = 0
    line1 = ""
    line2 = ""
    line3 = ""
    line4 = ""
    for line in iterable_read:
        if x == 0:
            line1 = line
        if x == 1:
            line2 = line
        if x == 2:
            line3 = line
        if x == 3:
            line4 = line
        x = x+1
    return "%s\n%s\n%s\n%s\n" % (line1, line2, line3, line4)

In [14]:
reads_tuple = indexed_raw_input.groupByKey().mapValues(lambda x: make_reads(x))

In [15]:
test = reads_tuple.take(10)

In [27]:
# Test get one record (brackets denote first tuple, second field because it is in format (0, read), (1, read) etc...)
print(test[0][1])

@SRR043378.1 618UWAAXX100205:6:1:995:4021/1
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
+
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



In [18]:
num_reads = reads_tuple.count()

In [28]:
print("Number of reads: %s" % num_reads)
# 16,924,141 reads

In [30]:
# Sort by key, which is just the line number, and then get the values, which is just the read.
readsRDD = reads_tuple.sortByKey().values()

In [ ]:
test2 = readsRDD.take(10)

In [40]:
print(test2[1])

In [43]:
readsRDD = readsRDD.repartition(2)

In [19]:
# TODO: split data into batches and send to bowtie 
# https://github.com/citiususc/SparkBWA/blob/master/src/main/java/com/github/sparkbwa/BwaAlignmentBase.java

In [ ]:
sc.stop()